In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import h5py

In [ ]:
h5f = h5py.File('../data/karim_stacked_fluxes', 'r')

In [ ]:
X_test = h5f['X_test']

In [ ]:
import tensorflow as tf
from keras.callbacks import EarlyStopping

In [ ]:
import cobra
%load_ext autoreload
import sys
if not '/home/nlarusstone/cf_fba' in sys.path:
    sys.path.append('/home/nlarusstone/cf_fba')
import src.utils as utils
import src.flux_utils as futils
import src.vae as model_builder
%autoreload 2

In [ ]:
df = pd.read_csv('../data/Karim_MetEng_2018_Figure2_Data.csv')
df.drop(columns=['Area_1', 'Area_2', 'Conc_1', 'Conc_2'], inplace=True)
n_experiments = df.shape[0]
df.head()

In [ ]:
#X_train, y_train, X_test, y_test, btol_col, cols = futils.read_data('../data/flux_samps_2k', scale='flux_zero')
y_vals = futils.scale_data(data=df['AVG.1'].values, scale_type='flux_zero', in_place=False)

In [ ]:
batch_size = 256
X_shape, n_experiments = X_train.shape[2], df.shape[0]
targets = tf.convert_to_tensor(y_vals, dtype=tf.float32)
latent_dim = 2
use_corr = True
scale = 'flux_zero'
vae, encoder, generator = model_builder.build_vae(X_shape, n_experiments, targets, btol_col, [1024, 256], 
                                                  latent_dim, batch_size, use_corr, scale)
es = EarlyStopping(patience=5)
lh = model_builder.LossHistory()
#with tf.Session(config=tf.ConfigProto(
#                    intra_op_parallelism_threads=32)) as sess:
#    K.set_session(sess)
hist = vae.fit(X_train,
        shuffle=True,
        epochs=20,
        batch_size=batch_size,
        validation_data=(X_test, None),
        callbacks=[es, lh])

In [ ]:
from keras.models import load_model
#enc = load_model('../models/encoder_epochs=300_batch=256_dimension=2_corr=True.h5')
enc = load_model('../models/encoder_epochs=100_batch=256_dimension=2_corr=True_scale=flux_zero.h5')
#gen = load_model('../models/generator_epochs=300_batch=256_dimension=2_corr=True.h5')
gen = load_model('../models/generator_epochs=100_batch=256_dimension=2_corr=True_scale=flux_zero.h5')

In [ ]:
test_enc = enc.predict(X_test)
test_dec = gen.predict(test_enc)
#dec_df = pd.DataFrame(data=test_dec[:, 0, :], columns=cols)

In [ ]:
model = cobra.io.read_sbml_model('../models/ecoli_cf_base.sbml')

In [ ]:
for j in range(41):
    a = pd.DataFrame(data=test_dec[:, j, :], columns=cols)
    a.mean().plot(kind='hist')
    print test_dec[:, j, btol_col].mean()
    plt.show()

In [ ]:
thresh = np.logspace(start=-4, stop=0, num=20)
for t in thresh:
    objs = []
    for i, row in df.iterrows():
        model_i = utils.add_reagents_to_model(model, row)
        dec_df = pd.DataFrame(data=test_dec[:, i, :], columns=cols)
        bad_cols = cols[dec_df.mean().abs() < t]
        model_i.remove_reactions(bad_cols)
        sol = model_i.optimize()
        objs.append(sol.objective_value)
    print t, scipy.stats.pearsonr(objs, df['AVG.1'])

In [ ]:
scale_type = ['norm', 'robust', 'maxabs', 'negone', 'zero']
dirs = ['flux', 'exp']

In [ ]:
import pickle
import itertools
def compare_losses(category, opts):
    cats = ['epochs', 'batch', 'dimension', 'corr', 'scale']
    dvs = [100, 256, 2, True, 'flux_zero']
    idx = cats.index(category)
    del cats[idx]
    del dvs[idx]
    defdict = dict(zip(cats, dvs))
    for opt in opts:
        curdict = defdict.copy()
        curdict.update({category: opt})
        with open('../models/losses_epochs={epochs}_batch={batch}_dimension={dimension}_corr={corr}_scale={scale}.h5'.format(
            **curdict
        ), 'r') as f:
            losses = pickle.load(f)
            print opt, [ls[-1] for k, ls in losses.items()]
#compare_losses('scale', ["{0}_{1}".format(d, st) for d, st in itertools.product(dirs, scale_type)])
compare_losses('dimension', [2, 3, 5, 7, 10])

In [ ]:
a = pd.DataFrame(data=test_dec[~high_cols, i, :], columns=cols)
print a.shape
a.head()

In [ ]:
diff = pd.DataFrame(data=X_test[high_cols, 0, :], columns=cols)
diff.head()

In [ ]:
a.mean().plot(kind='hist')

In [ ]:
diff.mean().plot(kind='hist')

In [ ]:
x_test_encoded = test_enc

def get_rct(df, rct, y_test):
    y_new = []
    for ind in y_test:
        y_new.append(df[rct][ind])
    return y_new

cmap = cm.get_cmap('plasma', 41)
#for j in range(4):
j = 0

high_cols = (test_enc[:, j, 1] > 0)
print sum(high_cols)
xmin, xmax = np.amin(x_test_encoded[:, j, 0]), np.amax(x_test_encoded[:, j, 0])
ymin, ymax = np.amin(x_test_encoded[:, j, 1]), np.amax(x_test_encoded[:, j, 1])
x_diff = (xmax - xmin) / 10.0
y_diff = (ymax - ymin) / 10.0
#for col in df.columns[4:]:
plt.figure(figsize=(10, 10))
plt.scatter(x_test_encoded[:, j, 0], x_test_encoded[:, j, 1], c=high_cols)#get_rct(df, col, y_test), cmap=cmap)
plt.xlim((xmin - x_diff, xmax + x_diff))
plt.ylim((ymin - y_diff, ymax + y_diff))
plt.title(j)
#plt.colorbar()
plt.show()